In [2]:
#################################################################################
# WaterTAP Copyright (c) 2020-2023, The Regents of the University of California,
# through Lawrence Berkeley National Laboratory, Oak Ridge National Laboratory,
# National Renewable Energy Laboratory, and National Energy Technology
# Laboratory (subject to receipt of any required approvals from the U.S. Dept.
# of Energy). All rights reserved.
#
# Please see the files COPYRIGHT.md and LICENSE.md for full copyright and license
# information, respectively. These files are also available online at the URL
# "https://github.com/watertap-org/watertap/"
#################################################################################

from pyomo.environ import (
    ConcreteModel,
    value,
    Constraint,
    Objective,
    Var,
    Expression,
    TransformationFactory,
    units as pyunits,
    check_optimal_termination,
    assert_optimal_termination,
)
from pyomo.network import Arc, SequentialDecomposition

import pyomo.environ as pyo
from pyomo.environ import units
import pyomo.util.infeasible as infeas
from idaes.core import FlowsheetBlock
from idaes.core.solvers import get_solver
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state
from idaes.models.unit_models import Feed, Separator, Mixer, Product
from idaes.models.unit_models.translator import Translator
from idaes.models.unit_models.separator import SplittingType
from idaes.models.unit_models.mixer import MomentumMixingType
from idaes.models.unit_models.heat_exchanger import (
    HeatExchanger,
    HeatExchangerFlowPattern,
)
from idaes.core import UnitModelCostingBlock
import idaes.core.util.scaling as iscale
import idaes.logger as idaeslog

from watertap.unit_models.mvc.components import Evaporator, Compressor, Condenser
from watertap.unit_models.mvc.components.lmtd_chen_callback import (
    delta_temperature_chen_callback,
)
from watertap.unit_models.pressure_changer import Pump
import watertap.property_models.seawater_prop_pack as props_sw
import watertap.property_models.water_prop_pack as props_w
from watertap.costing import WaterTAPCosting
import math
import csv



def main():
    
    ut = 1 #utilization parameter
    # build, set operating conditions, initialize for simulation
    m = build()
    set_operating_conditions(m, ut)
    add_Q_ext(m, time_point=m.fs.config.time)
    initialize_system(m)
    # rescale costs after initialization because scaling depends on flow rates
    scale_costs(m)
    fix_outlet_pressures(m)  # outlet pressure are initially unfixed for initialization

    # set up for minimizing Q_ext in first solve
    # should be 1 DOF because Q_ext is unfixed
    print("DOF after initialization: ", degrees_of_freedom(m))
    m.fs.objective = Objective(expr=m.fs.Q_ext[0])

    print("\n***---First solve - simulation results---***")
    solver = get_solver()
    results = solve(m, solver=solver, tee=False)
    print("Termination condition: ", results.solver.termination_condition)
    assert_optimal_termination(results)
    display_metrics(m)
    display_design(m)

    print("\n***---Second solve - optimization results---***")
    m.fs.Q_ext[0].fix(0)  # no longer want external heating in evaporator
    del m.fs.objective

    set_up_optimization(m)
    results = solve(m, solver=solver, tee=False)
    display_metrics(m)
    display_design(m)
    print("Termination condition: ", results.solver.termination_condition)
    save_result_to_csv(m, ut)
    
    print("\n***---Third solve - optimization results---***")
    prommis_costing(m, ut)
    del m.fs.objective
    set_up_prommis_optimization(m)
    results = solve(m, solver=solver, tee=False)

    display_metrics(m)
    display_design(m)
    
    print("Termination condition: ", results.solver.termination_condition)

    
    prommis_to_csv(m)


def build():
    # flowsheet set up
    m = ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)

    # Properties
    m.fs.properties_feed = props_sw.SeawaterParameterBlock()
    m.fs.properties_vapor = props_w.WaterParameterBlock()

    # Unit models
    m.fs.feed = Feed(property_package=m.fs.properties_feed)

    m.fs.pump_feed = Pump(property_package=m.fs.properties_feed)

    m.fs.separator_feed = Separator(
        property_package=m.fs.properties_feed,
        outlet_list=["hx_distillate_cold", "hx_brine_cold"],
        split_basis=SplittingType.totalFlow,
    )

    m.fs.hx_distillate = HeatExchanger(
        hot_side_name="hot",
        cold_side_name="cold",
        hot={"property_package": m.fs.properties_feed, "has_pressure_change": True},
        cold={"property_package": m.fs.properties_feed, "has_pressure_change": True},
        delta_temperature_callback=delta_temperature_chen_callback,
        flow_pattern=HeatExchangerFlowPattern.countercurrent,
    )
    # Set lower bound of approach temperatures
    m.fs.hx_distillate.delta_temperature_in.setlb(2)
    m.fs.hx_distillate.delta_temperature_out.setlb(2)
    m.fs.hx_distillate.area.setlb(1)

    m.fs.hx_brine = HeatExchanger(
        hot_side_name="hot",
        cold_side_name="cold",
        hot={"property_package": m.fs.properties_feed, "has_pressure_change": True},
        cold={"property_package": m.fs.properties_feed, "has_pressure_change": True},
        delta_temperature_callback=delta_temperature_chen_callback,
        flow_pattern=HeatExchangerFlowPattern.countercurrent,
    )
    # Set lower bound of approach temperatures
    m.fs.hx_brine.delta_temperature_in.setlb(2)
    m.fs.hx_brine.delta_temperature_out.setlb(2)
    m.fs.hx_brine.area.setlb(1)

    m.fs.mixer_feed = Mixer(
        property_package=m.fs.properties_feed,
        momentum_mixing_type=MomentumMixingType.equality,
        inlet_list=["hx_distillate_cold", "hx_brine_cold"],
    )
    m.fs.mixer_feed.pressure_equality_constraints[0, 2].deactivate()

    m.fs.evaporator = Evaporator(
        property_package_feed=m.fs.properties_feed,
        property_package_vapor=m.fs.properties_vapor,
    )

    m.fs.compressor = Compressor(property_package=m.fs.properties_vapor)

    m.fs.condenser = Condenser(property_package=m.fs.properties_vapor)

    m.fs.tb_distillate = Translator(
        inlet_property_package=m.fs.properties_vapor,
        outlet_property_package=m.fs.properties_feed,
    )

    # Translator block to convert distillate exiting condenser from water to seawater prop pack
    @m.fs.tb_distillate.Constraint()
    def eq_flow_mass_comp(blk):
        return (
            blk.properties_in[0].flow_mass_phase_comp["Liq", "H2O"]
            == blk.properties_out[0].flow_mass_phase_comp["Liq", "H2O"]
        )

    @m.fs.tb_distillate.Constraint()
    def eq_temperature(blk):
        return blk.properties_in[0].temperature == blk.properties_out[0].temperature

    @m.fs.tb_distillate.Constraint()
    def eq_pressure(blk):
        return blk.properties_in[0].pressure == blk.properties_out[0].pressure

    m.fs.pump_brine = Pump(property_package=m.fs.properties_feed)

    m.fs.pump_distillate = Pump(property_package=m.fs.properties_feed)

    m.fs.distillate = Product(property_package=m.fs.properties_feed)

    m.fs.brine = Product(property_package=m.fs.properties_feed)

    # Connections and connect condenser and evaporator
    m.fs.s01 = Arc(source=m.fs.feed.outlet, destination=m.fs.pump_feed.inlet)
    m.fs.s02 = Arc(source=m.fs.pump_feed.outlet, destination=m.fs.separator_feed.inlet)
    m.fs.s03 = Arc(
        source=m.fs.separator_feed.hx_distillate_cold,
        destination=m.fs.hx_distillate.cold_inlet,
    )
    m.fs.s04 = Arc(
        source=m.fs.separator_feed.hx_brine_cold, destination=m.fs.hx_brine.cold_inlet
    )
    m.fs.s05 = Arc(
        source=m.fs.hx_distillate.cold_outlet,
        destination=m.fs.mixer_feed.hx_distillate_cold,
    )
    m.fs.s06 = Arc(
        source=m.fs.hx_brine.cold_outlet, destination=m.fs.mixer_feed.hx_brine_cold
    )
    m.fs.s07 = Arc(
        source=m.fs.mixer_feed.outlet, destination=m.fs.evaporator.inlet_feed
    )
    m.fs.s08 = Arc(
        source=m.fs.evaporator.outlet_vapor, destination=m.fs.compressor.inlet
    )
    m.fs.s09 = Arc(source=m.fs.compressor.outlet, destination=m.fs.condenser.inlet)
    m.fs.s10 = Arc(
        source=m.fs.evaporator.outlet_brine, destination=m.fs.pump_brine.inlet
    )
    m.fs.s11 = Arc(source=m.fs.pump_brine.outlet, destination=m.fs.hx_brine.hot_inlet)
    m.fs.s12 = Arc(source=m.fs.hx_brine.hot_outlet, destination=m.fs.brine.inlet)
    m.fs.s13 = Arc(source=m.fs.condenser.outlet, destination=m.fs.tb_distillate.inlet)
    m.fs.s14 = Arc(
        source=m.fs.tb_distillate.outlet, destination=m.fs.pump_distillate.inlet
    )
    m.fs.s15 = Arc(
        source=m.fs.pump_distillate.outlet, destination=m.fs.hx_distillate.hot_inlet
    )
    m.fs.s16 = Arc(
        source=m.fs.hx_distillate.hot_outlet, destination=m.fs.distillate.inlet
    )

    TransformationFactory("network.expand_arcs").apply_to(m)

    m.fs.evaporator.connect_to_condenser(m.fs.condenser)

    # Add costing
    add_costing(m)

    # Add recovery ratio
    m.fs.recovery = Var(m.fs.config.time, initialize=0.5, bounds=(0, 1))
    m.fs.recovery_equation = Constraint(
        expr=m.fs.evaporator.properties_vapor[0].flow_mass_phase_comp["Vap", "H2O"]
        == m.fs.recovery[0]
        * (
            m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"]
            + m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "TDS"]
        )
    )

    # Make split ratio equal to recovery
    m.fs.split_ratio_recovery_equality = Constraint(
        expr=m.fs.separator_feed.split_fraction[0, "hx_distillate_cold"]
        == m.fs.recovery[0]
    )

    # Scaling
    # properties
    m.fs.properties_feed.set_default_scaling(
        "flow_mass_phase_comp", 1, index=("Liq", "H2O")
    )
    m.fs.properties_feed.set_default_scaling(
        "flow_mass_phase_comp", 1e2, index=("Liq", "TDS")
    )
    m.fs.properties_vapor.set_default_scaling(
        "flow_mass_phase_comp", 1, index=("Vap", "H2O")
    )
    m.fs.properties_vapor.set_default_scaling(
        "flow_mass_phase_comp", 1, index=("Liq", "H2O")
    )

    # unit model values
    # pumps
    iscale.set_scaling_factor(m.fs.pump_feed.control_volume.work, 1e-3)
    iscale.set_scaling_factor(m.fs.pump_brine.control_volume.work, 1e-3)
    iscale.set_scaling_factor(m.fs.pump_distillate.control_volume.work, 1e-3)

    # distillate HX
    iscale.set_scaling_factor(m.fs.hx_distillate.hot.heat, 1e-3)
    iscale.set_scaling_factor(m.fs.hx_distillate.cold.heat, 1e-3)
    iscale.set_scaling_factor(
        m.fs.hx_distillate.overall_heat_transfer_coefficient, 1e-3
    )

    iscale.set_scaling_factor(m.fs.hx_distillate.area, 1e-1)
    iscale.constraint_scaling_transform(
        m.fs.hx_distillate.cold_side.pressure_balance[0], 1e-5
    )
    iscale.constraint_scaling_transform(
        m.fs.hx_distillate.hot_side.pressure_balance[0], 1e-5
    )

    # brine HX
    iscale.set_scaling_factor(m.fs.hx_brine.hot.heat, 1e-3)
    iscale.set_scaling_factor(m.fs.hx_brine.cold.heat, 1e-3)
    iscale.set_scaling_factor(m.fs.hx_brine.overall_heat_transfer_coefficient, 1e-3)
    iscale.set_scaling_factor(m.fs.hx_brine.area, 1e-1)
    iscale.constraint_scaling_transform(
        m.fs.hx_brine.cold_side.pressure_balance[0], 1e-5
    )
    iscale.constraint_scaling_transform(
        m.fs.hx_brine.hot_side.pressure_balance[0], 1e-5
    )

    # evaporator
    iscale.set_scaling_factor(m.fs.evaporator.area, 1e-3)
    iscale.set_scaling_factor(m.fs.evaporator.U, 1e-3)
    iscale.set_scaling_factor(m.fs.evaporator.delta_temperature_in, 1e-1)
    iscale.set_scaling_factor(m.fs.evaporator.delta_temperature_out, 1e-1)
    iscale.set_scaling_factor(m.fs.evaporator.lmtd, 1e-1)

    # compressor
    iscale.set_scaling_factor(m.fs.compressor.control_volume.work, 1e-6)

    # condenser
    iscale.set_scaling_factor(m.fs.condenser.control_volume.heat, 1e-6)

    # calculate and propagate scaling factors
    iscale.calculate_scaling_factors(m)

    return m


def add_Q_ext(m, time_point=None):
    # Allows additional heat to be added to evaporator so that an initial feasible solution can be found as a starting
    # guess for optimization in case physically infeasible simulation is proposed

    if time_point is None:
        time_point = m.fs.config.time
    m.fs.Q_ext = Var(time_point, initialize=0, units=pyunits.J / pyunits.s)
    m.fs.Q_ext[0].setlb(0)
    m.fs.evaporator.eq_energy_balance.deactivate()
    m.fs.evaporator.eq_energy_balance_with_additional_Q = Constraint(
        expr=m.fs.evaporator.heat_transfer
        + m.fs.Q_ext[0]
        + m.fs.evaporator.properties_feed[0].enth_flow
        == m.fs.evaporator.properties_brine[0].enth_flow
        + m.fs.evaporator.properties_vapor[0].enth_flow_phase["Vap"]
    )
    iscale.set_scaling_factor(m.fs.Q_ext, 1e-6)


def add_costing(m):
    m.fs.costing = WaterTAPCosting()
    m.fs.pump_feed.costing = UnitModelCostingBlock(flowsheet_costing_block=m.fs.costing)
    m.fs.pump_distillate.costing = UnitModelCostingBlock(
        flowsheet_costing_block=m.fs.costing
    )
    m.fs.pump_brine.costing = UnitModelCostingBlock(
        flowsheet_costing_block=m.fs.costing
    )
    m.fs.hx_distillate.costing = UnitModelCostingBlock(
        flowsheet_costing_block=m.fs.costing
    )
    m.fs.hx_brine.costing = UnitModelCostingBlock(flowsheet_costing_block=m.fs.costing)
    m.fs.mixer_feed.costing = UnitModelCostingBlock(
        flowsheet_costing_block=m.fs.costing
    )
    m.fs.evaporator.costing = UnitModelCostingBlock(
        flowsheet_costing_block=m.fs.costing
    )
    m.fs.compressor.costing = UnitModelCostingBlock(
        flowsheet_costing_block=m.fs.costing
    )

    m.fs.costing.cost_process()
    m.fs.costing.add_annual_water_production(m.fs.distillate.properties[0].flow_vol)
    m.fs.costing.add_LCOW(m.fs.distillate.properties[0].flow_vol)
    m.fs.costing.add_specific_energy_consumption(m.fs.distillate.properties[0].flow_vol)
    m.fs.costing.base_currency = pyo.units.USD_2020


def set_operating_conditions(m, ut):
    # Feed inlet
    m.fs.feed.properties[0].flow_vol_phase["Liq"].fix(0.014877*1)                # volumetric flow rate (m3/s), equal to 235.8 gpm
    m.fs.feed.properties[0].conc_mass_phase_comp["Liq", "TDS"].fix(99.304)        # conc in g/L #base 99.304
#    m.fs.feed.properties[0].mass_frac_phase_comp["Liq", "TDS"].fix(0.093165) # mass frac, base 93165 ppm
#     m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"].fix(40)
#     m.fs.feed.properties[0].mass_frac_phase_comp["Liq", "TDS"].fix(0.1)
    # m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "TDS"].fix(4)
    m.fs.feed.properties[0].temperature.fix(273.15 + 25)
    m.fs.feed.properties[0].pressure.fix(101325)

    m.fs.recovery[0].fix(0.5)

    # Feed pump
    m.fs.pump_feed.efficiency_pump.fix(0.8)
    m.fs.pump_feed.control_volume.deltaP[0].fix(7e3)

    # Separator
    m.fs.separator_feed.split_fraction[0, "hx_distillate_cold"] = m.fs.recovery[0].value

    # Distillate HX
    m.fs.hx_distillate.overall_heat_transfer_coefficient.fix(2e3) #base 2000
    #m.fs.hx_distillate.area.fix(120)
    m.fs.hx_distillate.area = 120 #guess
    m.fs.hx_distillate.area.setub(2000)
    m.fs.hx_distillate.cold.deltaP[0].fix(7e3)
    m.fs.hx_distillate.hot.deltaP[0].fix(7e3)

    # Brine HX
    m.fs.hx_brine.overall_heat_transfer_coefficient.fix(2e3) #base 2000
    #m.fs.hx_brine.area.fix(100)
    m.fs.hx_brine.area = 100 #guess
    m.fs.hx_brine.area.setub(2000)
    m.fs.hx_brine.cold.deltaP[0].fix(7e3)
    m.fs.hx_brine.hot.deltaP[0].fix(7e3)

    # Evaporator
    m.fs.evaporator.inlet_feed.temperature[0] = 50 + 273.15  # provide guess
    m.fs.evaporator.outlet_brine.temperature[0].fix(72 + 273.15)
    m.fs.evaporator.U.fix(3e3)  # W/K-m^2 #base 3000
    m.fs.evaporator.area.setub(1e4)  # m^2

    # Compressor
    m.fs.compressor.pressure_ratio.fix(1.4)
    m.fs.compressor.efficiency.fix(0.7)

    # Brine pump
    m.fs.pump_brine.efficiency_pump.fix(0.8)
    m.fs.pump_brine.control_volume.deltaP[0].fix(4e4)

    # Distillate pump
    m.fs.pump_distillate.efficiency_pump.fix(0.8)
    m.fs.pump_distillate.control_volume.deltaP[0].fix(4e4)

    # Fix 0 TDS
    m.fs.tb_distillate.properties_out[0].flow_mass_phase_comp["Liq", "TDS"].fix(1e-5)

    # Costing
    eleccost = 8.09/100 #$/kWh, industrial, October 2023
    cepci2023 = 793.3 #September 2023
    cepci2020 = 596.2
    cepci2001 = 394.2
    default = 0
    if default == 0:
        m.fs.costing.high_pressure_pump.cost.fix(2.2)
        m.fs.costing.factor_total_investment.fix(3.4)
        m.fs.costing.electricity_cost.fix(eleccost)  
        m.fs.costing.heat_exchanger.material_factor_cost.fix(2.9) #base 2.9
        m.fs.costing.heat_exchanger.unit_cost.fix(300*cepci2023/cepci2020) #base 300 
        m.fs.costing.evaporator.unit_cost.fix(1000*cepci2023/cepci2020) #base 1000
        m.fs.costing.evaporator.material_factor_cost.fix(2.9) #base 2.9
        m.fs.costing.compressor.unit_cost.fix(1 * 7364*cepci2023/cepci2020)
        m.fs.costing.factor_maintenance_labor_chemical.fix(0.043) #base 0.043
        m.fs.costing.factor_capital_annualization.fix(0.0769)
        m.fs.costing.utilization_factor.fix(ut) #base 1
    else:
        m.fs.costing.electricity_cost.fix(eleccost)  
        m.fs.costing.compressor.unit_cost.fix(1 * 7364*cepci2023/cepci2020)
        m.fs.costing.factor_total_investment.fix(2)
        m.fs.costing.factor_maintenance_labor_chemical.fix(0.03)
        m.fs.costing.factor_capital_annualization.fix(0.1)
        m.fs.costing.heat_exchanger.unit_cost.fix(300*cepci2023/cepci2020)
        m.fs.costing.evaporator.unit_cost.fix(1000*cepci2023/cepci2020)

    # Temperature bounds
    m.fs.evaporator.properties_vapor[0].temperature.setub(75 + 273.15)
    m.fs.compressor.control_volume.properties_out[0].temperature.setub(450)

    # check degrees of freedom
    print("DOF after setting operating conditions: ", degrees_of_freedom(m))


def initialize_system(m, solver=None):
    if solver is None:
        solver = get_solver()
    optarg = solver.options

    # Touch feed mass fraction property
    m.fs.feed.properties[0].mass_frac_phase_comp["Liq", "TDS"]
    solver.solve(m.fs.feed)

    # Propagate vapor flow rate based on given recovery
    m.fs.evaporator.properties_vapor[0].flow_mass_phase_comp[
        "Vap", "H2O"
    ] = m.fs.recovery[0] * (
        m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"]
        + m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "TDS"]
    )
    m.fs.evaporator.properties_vapor[0].flow_mass_phase_comp["Liq", "H2O"] = 0

    # Propagate brine salinity and flow rate
    m.fs.evaporator.properties_brine[0].mass_frac_phase_comp[
        "Liq", "TDS"
    ] = m.fs.feed.properties[0].mass_frac_phase_comp["Liq", "TDS"] / (
        1 - m.fs.recovery[0]
    )
    m.fs.evaporator.properties_brine[0].mass_frac_phase_comp["Liq", "H2O"] = (
        1 - m.fs.evaporator.properties_brine[0].mass_frac_phase_comp["Liq", "TDS"].value
    )
    m.fs.evaporator.properties_brine[0].flow_mass_phase_comp[
        "Liq", "TDS"
    ] = m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "TDS"]
    m.fs.evaporator.properties_brine[0].flow_mass_phase_comp["Liq", "H2O"] = (
        m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"]
        - m.fs.evaporator.properties_vapor[0].flow_mass_phase_comp["Vap", "H2O"]
    )

    # initialize feed pump
    propagate_state(m.fs.s01)
    m.fs.pump_feed.initialize(optarg=optarg)

    # initialize separator
    propagate_state(m.fs.s02)
    # Touch property for initialization
    m.fs.separator_feed.mixed_state[0].mass_frac_phase_comp["Liq", "TDS"]
    m.fs.separator_feed.split_fraction[0, "hx_distillate_cold"].fix(
        m.fs.recovery[0].value
    )
    m.fs.separator_feed.mixed_state.initialize(optarg=optarg)
    # Touch properties for initialization
    m.fs.separator_feed.hx_brine_cold_state[0].mass_frac_phase_comp["Liq", "TDS"]
    m.fs.separator_feed.hx_distillate_cold_state[0].mass_frac_phase_comp["Liq", "TDS"]
    m.fs.separator_feed.initialize(optarg=optarg)
    m.fs.separator_feed.split_fraction[0, "hx_distillate_cold"].unfix()

    # initialize distillate heat exchanger
    propagate_state(m.fs.s03)
    m.fs.hx_distillate.cold_outlet.temperature[
        0
    ] = m.fs.evaporator.inlet_feed.temperature[0].value
    m.fs.hx_distillate.cold_outlet.pressure[0] = m.fs.evaporator.inlet_feed.pressure[
        0
    ].value
    m.fs.hx_distillate.hot_inlet.flow_mass_phase_comp[0, "Liq", "H2O"] = (
        m.fs.evaporator.properties_vapor[0].flow_mass_phase_comp["Vap", "H2O"].value
    )
    m.fs.hx_distillate.hot_inlet.flow_mass_phase_comp[0, "Liq", "TDS"] = 1e-4
    m.fs.hx_distillate.hot_inlet.temperature[
        0
    ] = m.fs.evaporator.outlet_brine.temperature[0].value
    m.fs.hx_distillate.hot_inlet.pressure[0] = 101325
    m.fs.hx_distillate.initialize()

    # initialize brine heat exchanger
    propagate_state(m.fs.s04)
    m.fs.hx_brine.cold_outlet.temperature[0] = m.fs.evaporator.inlet_feed.temperature[
        0
    ].value
    m.fs.hx_brine.cold_outlet.pressure[0] = m.fs.evaporator.inlet_feed.pressure[0].value
    m.fs.hx_brine.hot_inlet.flow_mass_phase_comp[
        0, "Liq", "H2O"
    ] = m.fs.evaporator.properties_brine[0].flow_mass_phase_comp["Liq", "H2O"]
    m.fs.hx_brine.hot_inlet.flow_mass_phase_comp[
        0, "Liq", "TDS"
    ] = m.fs.evaporator.properties_brine[0].flow_mass_phase_comp["Liq", "TDS"]
    m.fs.hx_brine.hot_inlet.temperature[0] = m.fs.evaporator.outlet_brine.temperature[
        0
    ].value
    m.fs.hx_brine.hot_inlet.pressure[0] = 101325
    m.fs.hx_brine.initialize()

    # initialize mixer
    propagate_state(m.fs.s05)
    propagate_state(m.fs.s06)
    m.fs.mixer_feed.initialize()
    m.fs.mixer_feed.pressure_equality_constraints[0, 2].deactivate()

    # initialize evaporator
    propagate_state(m.fs.s07)
    m.fs.Q_ext[0].fix()
    m.fs.evaporator.properties_vapor[0].flow_mass_phase_comp["Vap", "H2O"].fix()
    # fixes and unfixes those values
    m.fs.evaporator.initialize(delta_temperature_in=60)
    m.fs.Q_ext[0].unfix()
    m.fs.evaporator.properties_vapor[0].flow_mass_phase_comp["Vap", "H2O"].unfix()

    # initialize compressor
    propagate_state(m.fs.s08)
    m.fs.compressor.initialize()

    # initialize condenser
    propagate_state(m.fs.s09)
    m.fs.condenser.initialize(heat=-m.fs.evaporator.heat_transfer.value)

    # initialize brine pump
    propagate_state(m.fs.s10)
    m.fs.pump_brine.initialize(optarg=optarg)

    # initialize distillate pump
    propagate_state(m.fs.s13)  # to translator block
    propagate_state(m.fs.s14)  # from translator block to pump
    m.fs.pump_distillate.control_volume.properties_in[
        0
    ].temperature = m.fs.condenser.control_volume.properties_out[0].temperature.value
    m.fs.pump_distillate.control_volume.properties_in[
        0
    ].pressure = m.fs.condenser.control_volume.properties_out[0].pressure.value
    m.fs.pump_distillate.initialize(optarg=optarg)

    # propagate brine state
    propagate_state(m.fs.s12)
    propagate_state(m.fs.s16)

    seq = SequentialDecomposition(tear_solver="cbc")
    seq.options.log_info = False
    seq.options.iterLim = 1

    def func_initialize(unit):
        if unit.local_name == "feed":
            pass
        elif unit.local_name == "condenser":
            unit.initialize(
                heat=-unit.flowsheet().evaporator.heat_transfer.value,
                optarg=solver.options,
            )
        elif unit.local_name == "evaporator":
            unit.flowsheet().Q_ext[0].fix()
            unit.properties_vapor[0].flow_mass_phase_comp["Vap", "H2O"].fix()
            unit.initialize(delta_temperature_in=60)
            unit.flowsheet().Q_ext[0].unfix()
            unit.properties_vapor[0].flow_mass_phase_comp["Vap", "H2O"].unfix()
        elif unit.local_name == "separator_feed":
            unit.split_fraction[0, "hx_distillate_cold"].fix(
                unit.flowsheet().recovery[0].value
            )
            unit.initialize()
            unit.split_fraction[0, "hx_distillate_cold"].unfix()
        elif unit.local_name == "mixer_feed":
            unit.initialize()
            unit.pressure_equality_constraints[0, 2].deactivate()
        else:
            unit.initialize()

    seq.run(m, func_initialize)

    m.fs.costing.initialize()

    solver.solve(m, tee=False)

    print("Initialization done")


def prommis_costing(m, ut):
    utilization = ut
    TASC_TOC_factor = 1.207
    CRF_factor = 0.0769
    solids_mass = 0 * 1.10231e-6 * 24 * 365 * utilization #ton/yr, from OLI
    m.fs.costing.total_equip = m.fs.pump_feed.costing.capital_cost+m.fs.pump_brine.costing.capital_cost+m.fs.pump_distillate.costing.capital_cost+m.fs.hx_brine.costing.capital_cost+m.fs.hx_distillate.costing.capital_cost+m.fs.evaporator.costing.capital_cost+m.fs.compressor.costing.capital_cost+m.fs.mixer_feed.costing.capital_cost


    #Calculate BEC

    m.fs.costing.total_BEC = Expression(expr=m.fs.costing.total_equip * 1.58)

    #Calculate ancillary costs
    m.fs.costing.piping_MandL = Expression(expr=m.fs.costing.total_BEC * 0.2)
    m.fs.costing.elec_MandL = Expression(expr=m.fs.costing.total_BEC * 0.2)
    m.fs.costing.instrument = Expression(expr=m.fs.costing.total_BEC * 0.08)
    m.fs.costing.plant_serv = Expression(expr=m.fs.costing.total_BEC * 0.1)

    m.fs.costing.ancillary_cost = Expression(expr=(m.fs.costing.piping_MandL + m.fs.costing.elec_MandL + m.fs.costing.instrument + m.fs.costing.plant_serv))

    #calculate building costs
    m.fs.costing.proc_build = Expression(expr=m.fs.costing.total_BEC * 0.4)
    m.fs.costing.aux_build = Expression(expr=m.fs.costing.total_BEC * 0.15)
    m.fs.costing.sic_build = Expression(expr=m.fs.costing.total_BEC * 0.1)

    m.fs.costing.building_cost = Expression(expr=(m.fs.costing.proc_build + m.fs.costing.aux_build + m.fs.costing.sic_build))

    #calculate EPCM costs
    m.fs.costing.eic = Expression(expr=m.fs.costing.total_BEC * 0.17)
    m.fs.costing.fec = Expression(expr=m.fs.costing.total_BEC * 0.12)
    m.fs.costing.pmcc = Expression(expr=m.fs.costing.total_BEC * 0.3)

    m.fs.costing.epcm_cost = Expression(expr=(m.fs.costing.eic+m.fs.costing.fec+m.fs.costing.pmcc))

    #Contingency costs
    m.fs.costing.contingency_cost = Expression(expr=(0.15*m.fs.costing.total_BEC))

    #Total installed costs
    m.fs.costing.TIC_cost = Expression(expr=(m.fs.costing.ancillary_cost+m.fs.costing.building_cost+m.fs.costing.epcm_cost+m.fs.costing.contingency_cost))

    #Total plant costs, total overnight cost
    m.fs.costing.TPC_cost = Expression(expr=(m.fs.costing.total_BEC+m.fs.costing.TIC_cost))
    m.fs.costing.TOC_cost = Expression(expr=(m.fs.costing.TPC_cost))

    #TASC and annualized capital costs


    m.fs.costing.TASC_cost = Expression(expr=(TASC_TOC_factor * m.fs.costing.TOC_cost))
    m.fs.costing.prommis_annualized_capital_cost = Expression(expr=(CRF_factor*m.fs.costing.TASC_cost))

    #Fixed operating costs

    #Labor costs (no technical labor)

    #Operating Labor
    N_operator = 2
    operator_cost = 24.81*1
    t_shift = 8 #hours per shift
    N_shifts = 3 #number of shifts per day
    N_days = 365*utilization
    labor_burden = 0.25
    
    #indices for operating labor from stlouisfed, https://fred.stlouisfed.org/series/ECIWAG

    eciq22022 = 154.0
    eci2023 = (159.4+161.1+162.7+164.4)/4

    eciratio = eci2023 / eciq22022

    m.fs.costing.prommis_operating_labor_cost = Expression(expr=(N_operator*operator_cost*(1+labor_burden)*t_shift*N_shifts*N_days*eciratio*units.USD_2018))

    #maintenance and material costs
    m.fs.costing.MM_cost = Expression(expr=0.02*m.fs.costing.TPC_cost)

    #QAQC costs
    m.fs.costing.QAQC_cost = Expression(expr=0.1*m.fs.costing.prommis_operating_labor_cost)

    #Admin labor costs
    m.fs.costing.Admin_labor_cost = Expression(expr=0.2*m.fs.costing.prommis_operating_labor_cost)

    #Patent costs are ignored (no sales yet)
    #Property taxes

    m.fs.costing.prop_tax_insurance_cost = Expression(expr=0.01*m.fs.costing.TPC_cost)

    #WaterTAP costs are included here for fixed operating costs
    #For this flowsheet, only membrane replacement is needed


    #Summing relevant costs
    m.fs.costing.prommis_fixed_operating_cost = Expression(expr=(m.fs.costing.prommis_operating_labor_cost+m.fs.costing.MM_cost+m.fs.costing.QAQC_cost+m.fs.costing.Admin_labor_cost+m.fs.costing.prop_tax_insurance_cost))

    #Variable operating costs
    #land costs are set to 0

    #Per resource (waste disposal, antiscalant, electricity)
    #Note that solid masses are taken from OLI (not calculated in WaterTAP)
    m.fs.costing.liquid_waste_resource_cost = Expression(expr=(value(units.convert(m.fs.brine.properties[0].flow_vol_phase["Liq"],to_units=units.m ** 3 / units.hr))*264.172/42*24*365*1.5*1*utilization*units.USD_2018))
    m.fs.costing.solid_waste_resource_cost = Expression(expr=solids_mass*1*units.USD_2018)
    m.fs.costing.VOP_resource_cost = Expression(expr=(m.fs.costing.aggregate_flow_costs["electricity"]*utilization+m.fs.costing.liquid_waste_resource_cost+m.fs.costing.solid_waste_resource_cost))

    m.fs.costing.plant_overhead_cost = Expression(expr=(0.2*(m.fs.costing.prommis_fixed_operating_cost+m.fs.costing.VOP_resource_cost)))

    m.fs.costing.prommis_variable_operating_cost = Expression(expr=(m.fs.costing.VOP_resource_cost+m.fs.costing.plant_overhead_cost))
    #Annualized costs, prommis LCOW

    m.fs.costing.prommis_annualized_cost = Expression(expr=(m.fs.costing.prommis_annualized_capital_cost+m.fs.costing.prommis_fixed_operating_cost+m.fs.costing.prommis_variable_operating_cost))

    m.fs.costing.prommis_LCOW = Expression(expr=(m.fs.costing.prommis_annualized_cost/
                                             (value(units.convert(m.fs.distillate.properties[0].flow_vol,
                                                                  to_units = units.m ** 3 / units.hr))*24*365*utilization)))

    
def fix_outlet_pressures(m):
    # The distillate outlet pressure remains unfixed so that there is not an implicit upper bound on the compressed vapor pressure

    # Unfix pump heads
    m.fs.pump_brine.control_volume.deltaP[0].unfix()
    # m.fs.pump_distillate.control_volume.deltaP[0].unfix()

    # Fix outlet pressures
    m.fs.brine.properties[0].pressure.fix(101325)
    # m.fs.distillate.properties[0].pressure.fix(101325)

    return


def calculate_cost_sf(cost):
    sf = 10 ** -(math.log10(abs(cost.value)))
    iscale.set_scaling_factor(cost, sf)


def scale_costs(m):
    calculate_cost_sf(m.fs.hx_distillate.costing.capital_cost)
    calculate_cost_sf(m.fs.hx_brine.costing.capital_cost)
    calculate_cost_sf(m.fs.mixer_feed.costing.capital_cost)
    calculate_cost_sf(m.fs.evaporator.costing.capital_cost)
    calculate_cost_sf(m.fs.compressor.costing.capital_cost)
    calculate_cost_sf(m.fs.costing.aggregate_capital_cost)
    calculate_cost_sf(m.fs.costing.aggregate_flow_costs["electricity"])
    calculate_cost_sf(m.fs.costing.total_capital_cost)
    calculate_cost_sf(m.fs.costing.maintenance_labor_chemical_operating_cost)
    calculate_cost_sf(m.fs.costing.total_operating_cost)

    iscale.calculate_scaling_factors(m)

    print("Scaled costs")


def solve(model, solver=None, tee=False, raise_on_failure=False):
    # ---solving---
    if solver is None:
        solver = get_solver()

    results = solver.solve(model, tee=tee)
    if check_optimal_termination(results):
        return results
    msg = (
        "The current configuration is infeasible. Please adjust the decision variables."
    )
    if raise_on_failure:
        raise RuntimeError(msg)
    else:
        print(msg)
        return results


def set_up_optimization(m):
    m.fs.objective = Objective(expr=m.fs.costing.LCOW)
    m.fs.Q_ext[0].fix(0)
    m.fs.evaporator.area.unfix()
    m.fs.evaporator.outlet_brine.temperature[0].unfix()
    m.fs.recovery[0].fix(0.5)
    m.fs.compressor.pressure_ratio.unfix()
    m.fs.hx_distillate.area.unfix()
    m.fs.hx_brine.area.unfix()

    print("DOF for optimization: ", degrees_of_freedom(m))

def set_up_prommis_optimization(m):
    m.fs.objective = Objective(expr=m.fs.costing.prommis_LCOW)
#     m.fs.Q_ext[0].fix(0)
#     m.fs.evaporator.area.unfix()
#     m.fs.evaporator.outlet_brine.temperature[0].unfix()
#     m.fs.recovery[0].fix(0.5)
#     m.fs.compressor.pressure_ratio.unfix()
#     m.fs.hx_distillate.area.unfix()
#     m.fs.hx_brine.area.unfix()

    print("DOF for optimization: ", degrees_of_freedom(m))
    
def display_metrics(m):
    print("\nSystem metrics")
    print(
        "Volumetric Feed flow rate:                           %.2f m^3/s"
        % (
            m.fs.feed.properties[0].flow_vol_phase["Liq"].value

        )
    )
    print("Feed: %.2f m3/h, %.0f ppm" %
          (value(units.convert(m.fs.feed.properties[0].flow_vol_phase["Liq"],
                               to_units=units.m ** 3 / units.hr)),
           value(m.fs.feed.properties[0].mass_frac_phase_comp["Liq", "TDS"]) * 1e6))
    print(
        "Feed pressure:                           %.2f kPa"
        % (
            m.fs.evaporator.properties_feed[0].pressure.value * 1e-3
        )
    )
    print("Brine: %.2f m3/h, %.0f ppm" %
          (value(units.convert(m.fs.brine.properties[0].flow_vol_phase["Liq"],
                               to_units=units.m ** 3 / units.hr)),
           value(m.fs.brine.properties[0].mass_frac_phase_comp["Liq", "TDS"]) * 1e6))

    print("Distillate: %.2f m3/h" %
          (value(units.convert(m.fs.distillate.properties[0].flow_vol_phase["Liq"],
                               to_units=units.m ** 3 / units.hr))))
    print("Recovery:                                 %.2f %%"
        % (m.fs.recovery[0].value * 100))
    print("Specific energy consumption:              %.2f kWh/m3"
        % value(m.fs.costing.specific_energy_consumption))
    print("Levelized cost of water:                  %.2f $/m3" % m.fs.costing.LCOW.value)
    print("Total capital cost:                 $ %.2f" % value(m.fs.costing.total_capital_cost))
    print("Pumps cost:                 $ %.2f" % (value(m.fs.pump_feed.costing.capital_cost)*3.4+value(m.fs.pump_brine.costing.capital_cost)*3.4+value(m.fs.pump_distillate.costing.capital_cost)*3.4))
    print("HX cost:                 $ %.2f" % (value(m.fs.hx_brine.costing.capital_cost)*3.4+value(m.fs.hx_distillate.costing.capital_cost)*3.4))
    print("Evap cost:                 $ %.2f" % (value(m.fs.evaporator.costing.capital_cost)*3.4))
    print("Comp cost:                 $ %.2f" % (value(m.fs.compressor.costing.capital_cost)*3.4))
    print("Mixer cost:                 $ %.2f" % (value(m.fs.mixer_feed.costing.capital_cost)*3.4))
    print("Total operating cost:                 $ %.2f" % value(m.fs.costing.total_operating_cost))
    print("MLC cost:                 $ %.2f" % value(m.fs.costing.maintenance_labor_chemical_operating_cost))
    print("Aggregate electricity cost: %.2f $" % value(m.fs.costing.aggregate_flow_costs["electricity"]))
    print("External Q:                               %.2f W" % m.fs.Q_ext[0].value)  # should be 0 for optimization


def display_design(m):
    print("\nState variables")
    print(
        "Preheated feed temperature:               %.2f K"
        % m.fs.evaporator.properties_feed[0].temperature.value
    )
    print(
        "Evaporator (brine, vapor) temperature:    %.2f K"
        % m.fs.evaporator.properties_brine[0].temperature.value
    )
    print(
        "Evaporator (brine, vapor) pressure:       %.2f kPa"
        % (m.fs.evaporator.properties_vapor[0].pressure.value * 1e-3)
    )
    print(
        "Compressed vapor temperature:             %.2f K"
        % m.fs.compressor.control_volume.properties_out[0].temperature.value
    )
    print(
        "Compressed vapor pressure:                %.2f kPa"
        % (m.fs.compressor.control_volume.properties_out[0].pressure.value * 1e-3)
    )
    print(
        "Condensed vapor temperature:              %.2f K"
        % m.fs.condenser.control_volume.properties_out[0].temperature.value
    )

    print("\nDesign variables")
    print(
        "Brine heat exchanger area:                %.2f m2" % m.fs.hx_brine.area.value
    )
    print(
        "Brine heat exchanger cost:               $ %.2f" % value(m.fs.hx_brine.costing.capital_cost)
    )
    print(
        "Distillate heat exchanger area:           %.2f m2"
        % m.fs.hx_distillate.area.value
    )
    print(
        "Compressor pressure ratio:                %.2f"
        % m.fs.compressor.pressure_ratio.value
    )
    print(
        "Compressor work:                %.2f W"
        % value(m.fs.compressor.control_volume.work[0])
    )
    print(
        "Compressor flow rate:                %.2f kg/s"
        % value(m.fs.compressor.control_volume.properties_in[0].flow_mass_phase_comp["Vap", "H2O"])
    )
    print(
        "Evaporator area:                          %.2f m2" % m.fs.evaporator.area.value
    )
    print(
        "Evaporator LMTD:                          %.2f K" % m.fs.evaporator.lmtd.value
    )


def save_result_to_csv(m, ut):
    outarray = []
    col2 = []
    col2.append("LCOW ($/m3)")
    col2.append(value(m.fs.costing.LCOW))
    outarray.append(col2)
    col2 = []
    col2.append("Total Capital Cost $")
    col2.append(value(m.fs.costing.total_capital_cost))
    outarray.append(col2)
    col2 = []
    col2.append("Feed Pump Cost $")
    col2.append(value(m.fs.pump_feed.costing.capital_cost)*3.4)
    outarray.append(col2)
    col2 = []
    col2.append("Brine Pump Cost $")
    col2.append(value(m.fs.pump_brine.costing.capital_cost)*3.4)
    outarray.append(col2)
    col2 = []
    col2.append("Distillate Pump Cost $")
    col2.append(value(m.fs.pump_distillate.costing.capital_cost)*3.4)
    outarray.append(col2)
    col2 = []
    col2.append("Brine HX Cost $")
    col2.append(value(m.fs.hx_brine.costing.capital_cost)*3.4)
    outarray.append(col2)
    col2 = []
    col2.append("Distillate HX Cost $")
    col2.append(value(m.fs.hx_distillate.costing.capital_cost)*3.4)
    outarray.append(col2)
    col2 = []
    col2.append("Evaporator/Condenser Cost $")
    col2.append(value(m.fs.evaporator.costing.capital_cost)*3.4)
    outarray.append(col2)
    col2 = []
    col2.append("Compressor Cost $")
    col2.append(value(m.fs.compressor.costing.capital_cost)*3.4)
    outarray.append(col2)
    col2 = []
    col2.append("Feed Mixer Cost $")
    col2.append(value(m.fs.mixer_feed.costing.capital_cost)*3.4)
    outarray.append(col2)
    col2 = []
    col2.append("Total Operating Cost $/yr")
    col2.append(value(m.fs.costing.total_operating_cost))
    outarray.append(col2)
    col2 = []
    col2.append("MLC Cost $/yr")
    col2.append(value(m.fs.costing.maintenance_labor_chemical_operating_cost))
    outarray.append(col2)
    col2 = []
    col2.append("Elec Cost $/yr")
    col2.append(value(m.fs.costing.aggregate_flow_costs["electricity"])*ut)
    outarray.append(col2)
    col2 = []
    col2.append("Evaporator Area m2")
    col2.append(m.fs.evaporator.area.value)
    outarray.append(col2)
    col2 = []
    col2.append("Hot Brine Temp K")
    col2.append(m.fs.evaporator.properties_brine[0].temperature.value)
    outarray.append(col2)
    col2 = []
    col2.append("Compressor Pressure Ratio")
    col2.append(m.fs.compressor.pressure_ratio.value)
    outarray.append(col2)
    col2 = []
    col2.append("Brine HX Area m2")
    col2.append(m.fs.hx_brine.area.value)
    outarray.append(col2)
    col2 = []
    col2.append("Distillate HX Area m2")
    col2.append(m.fs.hx_distillate.area.value)
    outarray.append(col2)
    col2 = []
    col2.append("Compressed Vapor Pressure kPa")
    col2.append(m.fs.compressor.control_volume.properties_out[0].pressure.value * 1e-3)
    outarray.append(col2)
    col2 = []
    col2.append("Compressor Outlet Temperature K")
    col2.append(m.fs.compressor.control_volume.properties_out[0].temperature.value)
    outarray.append(col2)
    
    
    with open('watertappw.csv','w',newline='') as out:
        write=csv.writer(out)
        for x in range(len(outarray)):
            write.writerow(outarray[x])

def prommis_to_csv(m):
    outarray = []
    col2 = []
    col2.append("Prommis LCOW ($/m3)")
    col2.append(value(m.fs.costing.prommis_LCOW))
    outarray.append(col2)
    col2 = []
    col2.append("Total As-Spent Capital Cost $")
    col2.append(value(m.fs.costing.TASC_cost))
    outarray.append(col2)
    col2 = []
    col2.append("Total Fixed Operating Cost $/yr")
    col2.append(value(m.fs.costing.prommis_fixed_operating_cost))
    outarray.append(col2)
    col2 = []
    col2.append("Total Operating Labor $/yr")
    col2.append((value(m.fs.costing.prommis_operating_labor_cost)))
    outarray.append(col2)
    col2 = []
    col2.append("Total Variable Operating Cost $/yr")
    col2.append(value(m.fs.costing.prommis_variable_operating_cost))
    outarray.append(col2)
    col2 = []
    col2.append("Liquid Waste Disposal Cost $/yr")
    col2.append(value(m.fs.costing.liquid_waste_resource_cost))
    outarray.append(col2)
    col2 = []
    col2.append("Plant Overhead Cost $/yr")
    col2.append(value(m.fs.costing.plant_overhead_cost))
    outarray.append(col2)
    
    
    with open('watertappwprommis.csv','w',newline='') as out:
        write=csv.writer(out)
        for x in range(len(outarray)):
            write.writerow(outarray[x])

        
if __name__ == "__main__":
    m = main()

2024-06-10 15:12:09 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
DOF after setting operating conditions:  2
2024-06-10 15:12:09 [INFO] idaes.init.fs.pump_feed.control_volume.properties_out: fs.pump_feed.control_volume.properties_out State Released.
2024-06-10 15:12:09 [INFO] idaes.init.fs.pump_feed.control_volume: Initialization Complete
2024-06-10 15:12:09 [INFO] idaes.init.fs.pump_feed.control_volume.properties_in: fs.pump_feed.control_volume.properties_in State Released.
2024-06-10 15:12:09 [INFO] idaes.init.fs.pump_feed: Initialization Complete: optimal - Optimal Solution Found
2024-06-10 15:12:09 [INFO] idaes.init.fs.separator_feed.mixed_state: fs.separator_feed.mixed_state State Released.
2024-06-10 15:12:09 [INFO] idaes.init.fs.separator_feed.hx_distillate_cold_state: fs.separator_feed.hx_distillate_cold_state State

2024-06-10 15:12:14 [INFO] idaes.init.fs.condenser.control_volume: Initialization Complete
2024-06-10 15:12:14 [INFO] idaes.init.fs.condenser.control_volume.properties_in: fs.condenser.control_volume.properties_in State Released.
2024-06-10 15:12:14 [INFO] idaes.init.fs.condenser: Initialization Complete: optimal - Optimal Solution Found
2024-06-10 15:12:14 [INFO] idaes.init.fs.separator_feed.hx_distillate_cold_state: fs.separator_feed.hx_distillate_cold_state State Released.
2024-06-10 15:12:14 [INFO] idaes.init.fs.separator_feed.hx_brine_cold_state: fs.separator_feed.hx_brine_cold_state State Released.
2024-06-10 15:12:14 [INFO] idaes.init.fs.separator_feed: Initialization Step 2 Complete: optimal - Optimal Solution Found
2024-06-10 15:12:14 [INFO] idaes.init.fs.separator_feed.mixed_state: fs.separator_feed.mixed_state State Released.
2024-06-10 15:12:14 [INFO] idaes.init.fs.tb_distillate.properties_out: fs.tb_distillate.properties_out State Released.
2024-06-10 15:12:14 [INFO] idaes

2024-06-10 15:12:18 [INFO] idaes.init.fs.tb_distillate.properties_in: fs.tb_distillate.properties_in State Released.
2024-06-10 15:12:18 [INFO] idaes.init.fs.pump_distillate.control_volume.properties_out: fs.pump_distillate.control_volume.properties_out State Released.
2024-06-10 15:12:18 [INFO] idaes.init.fs.pump_distillate.control_volume: Initialization Complete
2024-06-10 15:12:18 [INFO] idaes.init.fs.pump_distillate.control_volume.properties_in: fs.pump_distillate.control_volume.properties_in State Released.
2024-06-10 15:12:18 [INFO] idaes.init.fs.pump_distillate: Initialization Complete: optimal - Optimal Solution Found
2024-06-10 15:12:18 [INFO] idaes.init.fs.hx_distillate.hot_side.properties_out: fs.hx_distillate.hot_side.properties_out State Released.
2024-06-10 15:12:18 [INFO] idaes.init.fs.hx_distillate.hot_side: Initialization Complete
2024-06-10 15:12:18 [INFO] idaes.init.fs.hx_distillate.cold_side.properties_out: fs.hx_distillate.cold_side.properties_out State Released.
2